# APPLIED DATA SCIENCE - CAPSTONE

_Author: Yulia Lopez_

### Description: This notebook is for the "Applied Data Science Capstone" assignments of the IBM Data Science Course. 

<a href='#ass1'>Assignment 1</a>

<a href='#ass2'>Assignment 2 - Segmenting and Clustering Neighborhoods in Toronto</a>
* <a href='#ass2pt1'>Part 1 - scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe </a>
* <a href='#ass2pt2'>Part 2 - get and add the latitude and the longitude coordinates of each neighborhood. </a>
* <a href='#ass2pt3'>Part 3 - explore and cluster the neighborhoods in Toronto. </a>

<a id='ass1'></a>
#### Assignment 1: 
* Write some markdown to explain that this notebook will be mainly used for the capstone project.
* Import the pandas library as pd.
* Import the Numpy library as np.
* Print the following the statement: Hello Capstone Project Course! 

In [9]:
import pandas as pd
import numpy as np

In [ ]:
print('Hello Capstone Project Course!')

<a id='ass2'></a>
#### Assignment 2:  Segmenting and Clustering Neighborhoods in Toronto
* you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. 
* you will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format.
* replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

<a id='ass2pt1'></a>
## Part 1 -  scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe

In [78]:
#import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [79]:
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'xml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()] 
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [80]:
# Ignore cells with a borough that is Not assigned.
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [81]:
df.shape

(211, 3)

In [82]:
#More than one neighborhood can exist in one postal code area. 
df1=df.groupby("PostalCode").agg(lambda x:','.join(set(x)))
df1.shape #Check results. 

(103, 2)

In [83]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df1.loc[df1['Neighbourhood']=="Not assigned",'Neighbourhood']=df1.loc[df1['Neighbourhood']=="Not assigned",'Borough']
df1.shape #Check results. 

(103, 2)

<a id='ass2pt2'></a> 
## Part 2 -  get and add the latitude and the longitude coordinates of each neighborhood.

In [84]:
df_coo = pd.read_csv("http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv") #Use csv table
df_coo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [96]:
df1['Latitude']=df_coo['Latitude'].values
df1['Longitude']=df_coo['Longitude'].values
df1.head()

,Borough,Neighbourhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
M1E,Scarborough,"West Hill,Guildwood,Morningside",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [95]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(len(df1['Borough'].unique()),df1.shape[0]))

Toronto has 11 boroughs and 103 neighborhoods.


In [97]:
df1.shape # check

(103, 4)

<a id='ass2pt3'></a> 
## Part 3 -  Explore and cluster the neighborhoods in Toronto.

In [99]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

print('Libraries imported.')

Libraries imported.


In [109]:
#Create a map of Toronto with neighborhoods.
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.4)

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [110]:

# add markers to map
for lat, lng, borough, neighbourhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [302]:
# get one test Neighbourhood
neigh_borough, neigh_name, neigh_lat, neigh_long = df1.iloc[0]
print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_lat, 
                                                               neigh_long))

Latitude and longitude values of Malvern,Rouge are 43.806686299999996, -79.19435340000001.


### Explore Neighbourhoods in Toronto

In [303]:
# function to repeat the same process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = getBuiltUrl(lat,lng,radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [263]:
neighborhoods = df1
toronto_venues = getNearbyVenues(names=df1['Neighbourhood'], 
                                   latitudes=df1['Latitude'], 
                                   longitudes=df1['Longitude'], radius=4000)

Malvern,Rouge
Highland Creek,Rouge Hill,Port Union
West Hill,Guildwood,Morningside
Woburn
Cedarbrae
Scarborough Village
Ionview,Kennedy Park,East Birchmount Park
Golden Mile,Oakridge,Clairlea
Cliffcrest,Cliffside,Scarborough Village West
Cliffside West,Birch Cliff
Scarborough Town Centre,Wexford Heights,Dorset Park
Maryvale,Wexford
Agincourt
Sullivan,Clarks Corners,Tam O'Shanter
L'Amoreaux East,Steeles East,Milliken,Agincourt North
L'Amoreaux West
Upper Rouge
Hillcrest Village
Oriole,Henry Farm,Fairview
Bayview Village
York Mills,Silver Hills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Downsview North,Wilson Heights,Bathurst Manor
Northwood Park,York University
Downsview East,CFB Toronto
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill,Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [264]:
print(toronto_venues.shape)
toronto_venues.head()

(10194, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern,Rouge",43.806686,-79.194353,Toronto Pan Am Sports Centre,43.790623,-79.193869,Athletics & Sports
1,"Malvern,Rouge",43.806686,-79.194353,African Rainforest Pavilion,43.817725,-79.183433,Zoo Exhibit
2,"Malvern,Rouge",43.806686,-79.194353,Toronto Zoo,43.820582,-79.181551,Zoo
3,"Malvern,Rouge",43.806686,-79.194353,Polar Bear Exhibit,43.823372,-79.185145,Zoo
4,"Malvern,Rouge",43.806686,-79.194353,Canadiana exhibit,43.817962,-79.193374,Zoo Exhibit


In [265]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,100,100,100,100,100,100
Bayview Village,100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
Caledonia-Fairbanks,100,100,100,100,100,100
Canada Post Gateway Processing Centre,100,100,100,100,100,100
Cedarbrae,69,69,69,69,69,69
Central Bay Street,100,100,100,100,100,100
"Chinatown,Kensington Market,Grange Park",100,100,100,100,100,100


In [266]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
print('There are {} uniques Neighborhoods.'.format(len(toronto_venues['Neighborhood'].unique())))

There are 272 uniques categories.
There are 103 uniques Neighborhoods.


### Analyze Each Neighborhoodds.

In [267]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Zoo Exhibit,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Leather Goods Store,Lingerie Store,Liquor Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [268]:
toronto_onehot.shape

(10194, 272)

In [269]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo Exhibit,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Leather Goods Store,Lingerie Store,Liquor Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Agincourt,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.010000,0.010000,0.000000,0.00,0.

In [270]:
toronto_grouped.shape

(103, 272)

In [271]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.09
1           Coffee Shop  0.07
2  Caribbean Restaurant  0.05
3           Supermarket  0.04
4     Indian Restaurant  0.04


----Bayview Village----
               venue  freq
0  Korean Restaurant  0.10
1             Bakery  0.06
2        Coffee Shop  0.06
3        Supermarket  0.04
4      Grocery Store  0.04


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.04
2                Hotel  0.04
3  Japanese Restaurant  0.03
4           Restaurant  0.03


----Business Reply Mail Processing Centre 969 Eastern----
            venue  freq
0            Café  0.08
1            Park  0.08
2     Coffee Shop  0.08
3         Brewery  0.05
4  Ice Cream Shop  0.04


----Caledonia-Fairbanks----
                venue  freq
0                Café  0.13
1         Coffee Shop  0.08
2  Italian Restaurant  0.08
3                 Bar  0.06
4      Ice Cream Shop  0.05


----Canada Pos

In [272]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [273]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Supermarket,Indian Restaurant,Bakery,Fast Food Restaurant,Bubble Tea Shop,Breakfast Spot,Steakhouse
1,Bayview Village,Korean Restaurant,Bakery,Coffee Shop,Middle Eastern Restaurant,Grocery Store,Supermarket,Chinese Restaurant,Bubble Tea Shop,Furniture / Home Store,Café
2,Berczy Park,Coffee Shop,Hotel,Café,Park,Farmers Market,Gym,Japanese Restaurant,Theater,Restaurant,Dessert Shop
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Park,Café,Brewery,Ice Cream Shop,Pizza Place,Gastropub,Vietnamese Restaurant,Beach,Bakery
4,Caledonia-Fairbanks,Café,Coffee Shop,Italian Restaurant,Bar,Ice Cream Shop,Brewery,Bakery,Park,Gastropub,Indian Restaurant
5,Canada Post Gateway Processing Centre,Coffee Shop,Hotel,Sandwich Place,Pharmacy,Japanese Restaurant,Middle Eastern Restaurant,Bank,Caribbean Restaurant,Breakfast Spot,Grocery Store
6,Cedarbrae,Coffee Shop,Indian Restaurant,Caribbean Restaurant,Chinese Restaurant,Park,Grocery Store,Asian Restaurant,Steakhouse,Breakfast Spot,Sandwich Place
7,Central Bay Street,Café,Coffee Shop,Hotel,Pizza Place,Restaurant,Theater,Beer Bar,Sandwich Place,Japanese Restaurant,Mexican Restaurant
8,"Chinatown,Kensington Market,Grange Park",Café,Coffee Shop,Bakery,Sandwich Place,Italian Restaurant,Pizza Place,Yoga Studio,Beer Bar,Park,Bar
9,Christie,Café,Bar,Italian Restaurant,Park,Vegetarian / Vegan Restaurant,Sandwich Place,Cocktail Bar,Asian Restaurant,Coffee Shop,Middle Eastern Restaurant


## Clustering

In [285]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 1, 3, 3, 2, 4, 1, 1, 3], dtype=int32)

In [291]:
toronto_grouped_clustering.shape

(103, 271)

In [292]:
toronto_merged = neighborhoods
toronto_merged = toronto_merged.rename(columns={'Neighbourhood':'Neighborhood'})
toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
M1E,Scarborough,"West Hill,Guildwood,Morningside",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [293]:
# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,,,,,
M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353,4,Zoo Exhibit,Coffee Shop,Sandwich Place,Fast Food Restaurant,Pharmacy,Pizza Place,Breakfast Spot,Burger Joint,Park,Chinese Restaurant
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,4,Coffee Shop,Fast Food Restaurant,Park,Pizza Place,Sandwich Place,Pharmacy,Breakfast Spot,Fried Chicken Joint,Burger Joint,Supermarket
M1E,Scarborough,"West Hill,Guildwood,Morningside",43.763573,-79.188711,1,Coffee Shop,Pizza Place,Fast Food Restaurant,Sandwich Place,Pharmacy,Burger Joint,Grocery Store,Fried Chicken Joint,Indian Restaurant,Liquor Store
M1G,Scarborough,Woburn,43.770992,-79.216917,3,Coffee Shop,Indian Restaurant,Pizza Place,Clothing Store,Caribbean Restaurant,Park,Liquor Store,Pharmacy,Bank,Gym
M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3,Coffee Shop,Indian Restaurant,Caribbean Restaurant,Chinese Restaurant,Park,Grocery Store,Asian Restaurant,Steakhouse,Breakfast Spot,Sandwich Place


In [304]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine clusters

## Cluster 0

In [295]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,
M2K,Bayview Village,Korean Restaurant,Bakery,Coffee Shop,Middle Eastern Restaurant,Grocery Store,Supermarket,Chinese Restaurant,Bubble Tea Shop,Furniture / Home Store,Café
M2L,"York Mills,Silver Hills",Burger Joint,Japanese Restaurant,Middle Eastern Restaurant,Bakery,Café,Coffee Shop,Steakhouse,Supermarket,Bubble Tea Shop,Grocery Store
M2M,"Newtonbrook,Willowdale",Korean Restaurant,Coffee Shop,Middle Eastern Restaurant,Supermarket,Grocery Store,Café,Chinese Restaurant,Bakery,Bubble Tea Shop,Burger Joint
M3M,Downsview Central,Clothing Store,Coffee Shop,Vietnamese Restaurant,Cosmetics Shop,Furniture / Home Store,Liquor Store,Jewelry Store,Bakery,Fried Chicken Joint,Athletics & Sports
M4N,Lawrence Park,Café,Italian Restaurant,Coffee Shop,Bakery,Park,Sushi Restaurant,Gym,Asian Restaurant,Sporting Goods Shop,Indian Restaurant
M4P,Davisville North,Coffee Shop,Park,Sushi Restaurant,Bakery,Italian Restaurant,Café,Gym,Japanese Restaurant,Burger Joint,Brewery
M5C,St. James Town,Coffee Shop,Park,Japanese Restaurant,Hotel,Restaurant,Café,Farmers Market,Pizza Place,Theater,Gym


### Cluster 1

In [296]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,
M1E,"West Hill,Guildwood,Morningside",Coffee Shop,Pizza Place,Fast Food Restaurant,Sandwich Place,Pharmacy,Burger Joint,Grocery Store,Fried Chicken Joint,Indian Restaurant,Liquor Store
M1L,"Golden Mile,Oakridge,Clairlea",Middle Eastern Restaurant,Coffee Shop,Burger Joint,Park,Sandwich Place,Supermarket,Bank,Chinese Restaurant,Breakfast Spot,Discount Store
M1M,"Cliffcrest,Cliffside,Scarborough Village West",Coffee Shop,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Park,Beer Store,Discount Store,Pizza Place,Bank,Pharmacy
M1P,"Scarborough Town Centre,Wexford Heights,Dorset...",Coffee Shop,Chinese Restaurant,Middle Eastern Restaurant,Supermarket,Pizza Place,Indian Restaurant,Fast Food Restaurant,Clothing Store,Gym,Breakfast Spot
M1V,"L'Amoreaux East,Steeles East,Milliken,Agincour...",Chinese Restaurant,Coffee Shop,Bakery,Bubble Tea Shop,Caribbean Restaurant,Vietnamese Restaurant,Supermarket,Noodle House,Indian Restaurant,Dessert Shop
M3C,"Flemingdon Park,Don Mills South",Middle Eastern Restaurant,Supermarket,Grocery Store,Gym / Fitness Center,Japanese Restaurant,Gym,Restaurant,Indian Restaurant,Park,Movie Theater
M3L,Downsview West,Coffee Shop,Pizza Place,Vietnamese Restaurant,Fast Food Restaurant,Sandwich Place,Bank,Pharmacy,Beer Store,Grocery Store,Turkish Restaurant
M4J,East Toronto,Café,Greek Restaurant,Park,Ice Cream Shop,Pizza Place,Coffee Shop,Brewery,Farmers Market,Gastropub,American Restaurant
M5P,"Forest Hill West,Forest Hill North",Café,Italian Restaurant,Coffee Shop,Park,Ice Cream Shop,Grocery Store,Indian Restaurant,Liquor Store,Middle Eastern Restaurant,Spa


## Cluster 2

In [297]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,
M1J,Scarborough Village,Coffee Shop,Pharmacy,Chinese Restaurant,Indian Restaurant,Fast Food Restaurant,Park,Gym,Sandwich Place,Clothing Store,Bank
M1R,"Maryvale,Wexford",Coffee Shop,Chinese Restaurant,Middle Eastern Restaurant,Burger Joint,Supermarket,Indian Restaurant,Burrito Place,Italian Restaurant,Fast Food Restaurant,Falafel Restaurant
M2N,Willowdale South,Korean Restaurant,Coffee Shop,Grocery Store,Middle Eastern Restaurant,Café,Thai Restaurant,Supermarket,Furniture / Home Store,Restaurant,Burger Joint
M2P,York Mills West,Coffee Shop,Sushi Restaurant,Grocery Store,Bakery,Café,Bubble Tea Shop,Park,Burger Joint,Supermarket,Korean Restaurant
M3B,Don Mills North,Middle Eastern Restaurant,Japanese Restaurant,Supermarket,Park,Ice Cream Shop,Liquor Store,Furniture / Home Store,Bakery,Gym / Fitness Center,Grocery Store
M4A,Victoria Village,Middle Eastern Restaurant,Indian Restaurant,Park,Supermarket,Liquor Store,Bakery,Gym,Gym / Fitness Center,Mediterranean Restaurant,Café
M4E,The Beaches,Park,Beach,Brewery,Café,Coffee Shop,Breakfast Spot,Bakery,Bar,Ice Cream Shop,Pub
M4K,"The Danforth West,Riverdale",Park,Coffee Shop,Café,Greek Restaurant,Japanese Restaurant,Ice Cream Shop,Italian Restaurant,Bakery,Brewery,Juice Bar
M4W,Rosedale,Coffee Shop,Park,Italian Restaurant,Japanese Restaurant,Pizza Place,Hotel,Café,Restaurant,Dance Studio,Bookstore


## Cluster 3

In [298]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,
M1G,Woburn,Coffee Shop,Indian Restaurant,Pizza Place,Clothing Store,Caribbean Restaurant,Park,Liquor Store,Pharmacy,Bank,Gym
M1H,Cedarbrae,Coffee Shop,Indian Restaurant,Caribbean Restaurant,Chinese Restaurant,Park,Grocery Store,Asian Restaurant,Steakhouse,Breakfast Spot,Sandwich Place
M1N,"Cliffside West,Birch Cliff",Park,Coffee Shop,Beach,Breakfast Spot,Pub,Bank,Bar,Thai Restaurant,BBQ Joint,Tea Room
M1S,Agincourt,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Supermarket,Indian Restaurant,Bakery,Fast Food Restaurant,Bubble Tea Shop,Breakfast Spot,Steakhouse
M1W,L'Amoreaux West,Chinese Restaurant,Bakery,Caribbean Restaurant,Bubble Tea Shop,Japanese Restaurant,Noodle House,Coffee Shop,Asian Restaurant,Tea Room,BBQ Joint
M1X,Upper Rouge,Zoo Exhibit,Grocery Store,Coffee Shop,Fast Food Restaurant,Pizza Place,Park,Sandwich Place,Skating Rink,Burger Joint,Golf Course
M2H,Hillcrest Village,Chinese Restaurant,Bakery,Coffee Shop,Japanese Restaurant,Caribbean Restaurant,Furniture / Home Store,Italian Restaurant,Breakfast Spot,Thai Restaurant,Fish Market
M2R,Willowdale West,Korean Restaurant,Coffee Shop,Middle Eastern Restaurant,Café,Supermarket,Grocery Store,Thai Restaurant,Bakery,Bubble Tea Shop,Seafood Restaurant
M3A,Parkwoods,Middle Eastern Restaurant,Coffee Shop,Japanese Restaurant,Chinese Restaurant,Bakery,Mediterranean Restaurant,Café,Supermarket,Italian Restaurant,Ice Cream Shop


## Cluster 4

In [301]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,
M1B,"Malvern,Rouge",Zoo Exhibit,Coffee Shop,Sandwich Place,Fast Food Restaurant,Pharmacy,Pizza Place,Breakfast Spot,Burger Joint,Park,Chinese Restaurant
M1C,"Highland Creek,Rouge Hill,Port Union",Coffee Shop,Fast Food Restaurant,Park,Pizza Place,Sandwich Place,Pharmacy,Breakfast Spot,Fried Chicken Joint,Burger Joint,Supermarket
M1K,"Ionview,Kennedy Park,East Birchmount Park",Coffee Shop,Chinese Restaurant,Middle Eastern Restaurant,Park,Burger Joint,Fast Food Restaurant,Indian Restaurant,Pharmacy,Supermarket,Burrito Place
M1T,"Sullivan,Clarks Corners,Tam O'Shanter",Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Bakery,Fast Food Restaurant,Pharmacy,Supermarket,Noodle House,Pizza Place,Korean Restaurant
M2J,"Oriole,Henry Farm,Fairview",Bakery,Chinese Restaurant,Caribbean Restaurant,Coffee Shop,Japanese Restaurant,Middle Eastern Restaurant,Burger Joint,Supermarket,Gym / Fitness Center,Furniture / Home Store
M3H,"Downsview North,Wilson Heights,Bathurst Manor",Coffee Shop,Clothing Store,Grocery Store,Restaurant,Café,Burger Joint,Sandwich Place,Bakery,Bagel Shop,Park
M3N,Downsview Northwest,Coffee Shop,Sandwich Place,Hotel,Restaurant,Fast Food Restaurant,Burger Joint,Pizza Place,Vietnamese Restaurant,Furniture / Home Store,Steakhouse
M4B,"Parkview Hill,Woodbine Gardens",Café,Gastropub,Bakery,Middle Eastern Restaurant,Brewery,Indian Restaurant,Thai Restaurant,Turkish Restaurant,Asian Restaurant,Bar
M4H,Thorncliffe Park,Park,Greek Restaurant,Café,Gastropub,Grocery Store,Italian Restaurant,Bakery,Coffee Shop,Brewery,Ice Cream Shop


# Thank you for your review!